<a href="https://colab.research.google.com/github/JacopoMangiavacchi/S4TF-EmbeddingMultiInput/blob/master/TF1_House.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow==1.12.0
!pip install coremltools

     |████████████████████████████████| 83.1MB 71kB/s 
     |████████████████████████████████| 3.1MB 21.8MB/s 
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstalled tensorflow-1.15.0
     |████████████████████████████████| 3.4MB 2.7MB/s 


In [2]:
import tensorflow as tf
print(tf.__version__)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or 

1.12.0


In [0]:
import tensorflow as tf
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, Input, Embedding, Flatten, concatenate
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import preprocessing
import coremltools

# Model

In [0]:
# Swift for TensorFlow Model

# struct RegressionModel: Module {
#     var numericalLayer = Dense<Float>(inputSize: 11, outputSize: 32, activation: relu)
#     var embedding1 = Embedding<Float>(vocabularySize: 2, embeddingSize: 2)
#     var embedding2 = Embedding<Float>(vocabularySize: 9, embeddingSize: 5)
#     var embeddingLayer = Dense<Float>(inputSize: (4 + 45), outputSize: 64, activation: relu)
#     var allInputConcatLayer = Dense<Float>(inputSize: (32 + 64), outputSize: 128, activation: relu)
#     var hiddenLayer = Dense<Float>(inputSize: 128, outputSize: 32, activation: relu)
#     var outputLayer = Dense<Float>(inputSize: 32, outputSize: 1)
    
#     @differentiable
#     func callAsFunction(_ input: MultiInputs<[Tensor<Float>], [Tensor<Int32>]>) -> Tensor<Float> {
#         let numericalInput = numericalLayer(input.numerical[0])
#         let embeddingOutput1 = embedding1(input.categorical[0])
#         let embeddingOutput1Reshaped = embeddingOutput1.reshaped(to: 
#             TensorShape([embeddingOutput1.shape[0], embeddingOutput1.shape[1] * embeddingOutput1.shape[2]]))
#         let embeddingOutput2 = embedding2(input.categorical[1])
#         let embeddingOutput2Reshaped = embeddingOutput2.reshaped(to: 
#             TensorShape([embeddingOutput2.shape[0], embeddingOutput2.shape[1] * embeddingOutput2.shape[2]]))
#         let embeddingConcat = Tensor<Float>(concatenating: [embeddingOutput1Reshaped, embeddingOutput2Reshaped], alongAxis: 1)
#         let embeddingInput = embeddingLayer(embeddingConcat)
#         let allConcat = Tensor<Float>(concatenating: [numericalInput, embeddingInput], alongAxis: 1)
#         return allConcat.sequenced(through: allInputConcatLayer, hiddenLayer, outputLayer)
#     }
# }

In [0]:
numerical_input = Input(shape=(11,))
categorical_input1 = Input(shape=(2,))
categorical_input2 = Input(shape=(9,))

numerical = Dense(32, activation='relu')(numerical_input)

embedding1 = Embedding(input_dim=2, output_dim=2, input_length=2)(categorical_input1)
embedding1 = Flatten()(embedding1)

embedding2 = Embedding(input_dim=9, output_dim=5, input_length=9)(categorical_input2)
embedding2 = Flatten()(embedding2)

embeddingConcat = concatenate([embedding1, embedding2], axis=1)
embedding = Dense(64, activation='relu')(embeddingConcat)

allConcat = concatenate([numerical, embedding], axis=1)
hidden = Dense(128, activation='relu')(allConcat)
hidden = Dense(32, activation='relu')(hidden)
output = Dense(1)(hidden)

keras_model = Model(inputs=[numerical_input, categorical_input1, categorical_input2], outputs=output)

In [9]:
keras_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, 2)            0                                            
__________________________________________________________________________________________________
input_9 (InputLayer)            (None, 9)            0                                            
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, 2, 2)         4           input_8[0][0]                    
__________________________________________________________________________________________________
embedding_6 (Embedding)         (None, 9, 5)         45          input_9[0][0]                    
____________________________________________________________________________________________

# Save and Export no Train Model

In [0]:
keras_model.save('keras_no_train_model.h5')

In [12]:
spec = mlmodel.get_spec()
spec

NameError: ignored